# Iterator

> Traversing data structures

In this lesson we'll implement the **iterator design pattern** by iterating through a binary tree like the following:

```
  1
 / \
2   3
```

Essentially, we'll have a tree composed of 3 elements, with a node of value 1 on top and two nodes connecting to it of values 2 and 3.

We will see how to iterate through it in 2 different ways:
1. The "canonical" approach using constructs such as iter.
2. By writing a function that will use the `yield` keyword.

Both of these approaches are equally valid; the main issue is that stateful iterators are typically very painful to do, as we shall soon see.

Let's begin by implementing a `Node` class. A `Node` instance has a value, as well as a left and a right child nodes. We'll also define a parent node.

In [ ]:
class Node:
    def __init__(self, value, left=None, right=None):
        self.right = right
        self.left = left
        self.value = value

        self.parent = None

        if left:
            self.left.parent = self
        if right:
            self.right.parent = self

Note that there are 3 ways of traversing this binary tree depending on the order in which you access the nodes:
* **In-order**: `2 -> 1 -> 3`
* **Pre-order**: `1 -> 2 -> 3`
* **Post-order**: `2 -> 3 -> 1`

We will stick to in-order iteration.

We will need to define a new class that will allow for in-order iteration; this `InOrderIterator` class will be defined by a `root` (the node we want to stick to always), a `current` (our current placement in the tree -> what makes our iterator a **stateful operator**), and since we're iterating in-order, we will need to navigate to the leftmost element first.

We will also implement the `__next__` function that will return each of the iterated elements. This function will make use of a special `yielded_start` flag that lets us know whether the intial value was yielded. The code for this function is quite complex because it needs to keep track of the state of the iterator, but it will navigate the tree in-order.

In [2]:
class InOrderIterator:
    def __init__(self, root):
        self.root = self.current = root
        self.yielded_start = False
        while self.current.left:
            self.current = self.current.left
            
    def __next__(self):
        if not self.yielded_start:
            self.yielded_start = True
            return self.current

        if self.current.right:
            self.current = self.current.right
            while self.current.left:
                self.current = self.current.left
            return self.current
        else:
            p = self.current.parent
            while p and self.current == p.right:
                self.current = p
                p = p.parent
            self.current = p
            if self.current:
                return self.current
            else:
                raise StopIteration

We also need to expose our iterator. Since we do not have a special class for binary trees, we will expose the iterator from the `Node` class, which we can do by implementing the `__iter__` function within `Node`.

In [3]:
class Node:
  def __init__(self, value, left=None, right=None):
    self.right = right
    self.left = left
    self.value = value

    self.parent = None

    if left:
      self.left.parent = self
    if right:
      self.right.parent = self

  def __iter__(self):
    return InOrderIterator(self)

We're finally ready to create our binary tree and iterate through it. We can do it in 2 ways:
* Iterate explicitly, by creating an `InORderIterator` instance imperatively and calling `next` to iterate.
* Iterate implicitly, by using a `for` loop, which will make use of both `__iter__` and `__next__`.

In [4]:
# let's create a tree
root = Node(1, Node(2), Node(3))

# iterate explicitly
it = iter(root)
print([next(it).value for x in range(3)]) # we know that there are 3 elements
print('---')

# iterate implicitly
for x in root:
    print(x.value)

[2, 1, 3]
---
2
1
3
